In [1]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings

def get_insights(question):
    # Load and process the CSV data
    loader = CSVLoader("Marks.csv")
    documents = loader.load()

    # Create embeddings
    embeddings = OllamaEmbeddings(model="llama2")

    chroma_db = Chroma.from_documents(
        documents, embeddings, persist_directory="./chroma_db"
    )
    chroma_db.persist()

    llm = Ollama(model="llama2")

    prompt_template = PromptTemplate(
        input_variables=["context"],
        template="Given this context: {context}, please directly answer the question: {question}.",
    )

    # Set up the question-answering chain
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=chroma_db.as_retriever(),
        chain_type_kwargs={"prompt": prompt_template},
    )
    print(chroma_db.as_retriever())
    result = qa_chain({"query": question})
    return result

In [2]:
get_insights("What is the average marks of students in the class?")

tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x112fa0bd0>


/Users/danielmoreno/NLP/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'What is the average marks of students in the class?',
 'result': 'The average marks of the students in the class is 76.'}

```csv
Student_Name,Subject,Marks
Jack,Math,90
John,Math,60
Mary,Math,70
Peter,Math,80
```

In [4]:
grades = [90, 60, 70, 80]
average = sum(grades) / len(grades)
print(f"The average marks of students in the class is {average}")

The average marks of students in the class is 75.0
